# Single Tool Use for Cohere on Bedrock Converse API

---

## Introduction
In this notebook, we demonstrate how to use the Bedrock Converse API with the `list_calendar_events` and `create_calendar_event` tools to book appointments. Booking the correct appointment requires the model to first check for an available slot by listing existing events, reasoning about the correct slot to book the new appointment and then finally invoking the right tool to create the calendar event. All code was orinally provided by Cohere and this notebook is a supplement of the original notebook but now instead uses Amazon Bedrock's Converse API.


Tool use allows customers to connect their large language models to external tools like search engines, APIs, functions, databases, etc. Learn more on Cohere's original cookbook for this notebook here: https://docs.cohere.com/page/basic-tool-use

This allows the customer's model to unlock a richer set of behaviors by leveraging data stored in tools, taking actions through APIs, interacting with a vector database, querying a search engine, etc.

This is particularly valuable for enterprise customers, since a lot of enterprise data lives in external sources.

---

## Getting Started

### Step 0: Install Dependencies

Here, we will install all the required dependencies to run this notebook.

In [ ]:
pip install boto3==1.34.120 --quiet

### Step 1: Select Model

Below sets default model to command R+ model with option to change to Command R

In [ ]:
DEFAULT_MODEL= "cohere.command-r-plus-v1:0"
COMMAND_R_PLUS = "cohere.command-r-plus-v1:0"
COMMAND_R = "cohere.command-r-v1:0"
model_id = DEFAULT_MODEL

Now lets import the required modules to run the notbook and set up the Bedrock client

In [ ]:
import boto3, json
bedrock_rt = boto3.client(service_name="bedrock-runtime", region_name = "us-east-1")

### Step 2: Create a mock database
First, we'll define the mock data that our tools will query. This data represents sales reports and a product catalog.

In [ ]:
# Mock database containing daily sales reports
sales_database = {
    '2023-09-28': {
        'total_sales_amount': 5000,
        'total_units_sold': 100,
    },
    '2023-09-29': {
        'total_sales_amount': 10000,
        'total_units_sold': 250,
    },
    '2023-09-30': {
        'total_sales_amount': 8000,
        'total_units_sold': 200,
    }
}

# Mock product catalog
product_catalog = {
    'Electronics': [
        {'product_id': 'E1001', 'name': 'Smartphone', 'price': 500, 'stock_level': 20},
        {'product_id': 'E1002', 'name': 'Laptop', 'price': 1000, 'stock_level': 15},
        {'product_id': 'E1003', 'name': 'Tablet', 'price': 300, 'stock_level': 25},
    ],
    'Clothing': [
        {'product_id': 'C1001', 'name': 'T-Shirt', 'price': 20, 'stock_level': 100},
        {'product_id': 'C1002', 'name': 'Jeans', 'price': 50, 'stock_level': 80},
        {'product_id': 'C1003', 'name': 'Jacket', 'price': 100, 'stock_level': 40},
    ]
}

Now, we'll define the tools that simulate querying this database.  
You could for example use the API of an enterprise sales platform.



In [ ]:
def query_daily_sales_report(day: str) -> dict:
    """
    Function to retrieve the sales report for the given day
    """
    report = sales_database.get(day, {})
    if report:
        return {
            'date': day,
            'summary': f"Total Sales Amount: {report['total_sales_amount']}, Total Units Sold: {report['total_units_sold']}"
        }
    else:
        return {'date': day, 'summary': 'No sales data available for this day.'}


def query_product_catalog(category: str) -> dict:
    """
    Function to retrieve products for the given category
    """
    products = product_catalog.get(category, [])
    return {
        'category': category,
        'products': products
    }


functions_map = {
    "query_daily_sales_report": query_daily_sales_report,
    "query_product_catalog": query_product_catalog
}

### Step 3 - Configure the request to the model

The developer provides a few things to the model:
- A preamble containing instructions about the task and the desired style for the output.
- The user request.
- A list of tools to the model.
- (Optionally) a chat history for the model to work with.


You can specify one or many tools to the model. Every tool needs to be described with a JSON schema, indicating the tool name, description, and parameters (code snippets below).

In our example, we provide two tools to the model: `daily_sales_report` and `product_catalog`.


Defining the tools using Bedrock APIs is a bit different than with Cohere's APIs. Here is an example of the same tools being define for Cohere's APIs: 

```
tools = [
    {
        "name": "query_daily_sales_report",
        "description": "Connects to a database to retrieve overall sales volumes and sales information for a given day.",
        "parameter_definitions": {
            "day": {
                "description": "Retrieves sales data for this day, formatted as YYYY-MM-DD.",
                "type": "str",
                "required": True
            }
        }
    },
    {
        "name": "query_product_catalog",
        "description": "Connects to a a product catalog with information about all the products being sold, including categories, prices, and stock levels.",
        "parameter_definitions": {
            "category": {
                "description": "Retrieves product information data for all products in this category.",
                "type": "str",
                "required": True
            }
        }
    }
]
```

Here is the same tools being defined for Bedrock APIs. You can see that the key differences are where the descriptions are defined and the toolspec key. We recommend defining tools this way in an array, appending each tool to keep the same structure.

In [ ]:
tool_config = {'tools': []}

tool_config['tools'].append({
    'toolSpec': {
        'name': 'query_daily_sales_report',
        'description': 'Connects to a database to retrieve overall sales volumes and sales information for a given day.',
        'inputSchema': {
            'json': {
                'type': 'object',
                'properties': {
                    'day': {
                        'type': 'string',
                        'description': 'Retrieves sales data for this day, formatted as YYYY-MM-DD.'
                    }
                },
                'required': [
                    'day'
                ]
            }
        }
    }
})


tool_config['tools'].append({
    'toolSpec': {
        'name': 'query_product_catalog',
        'description': 'Connects to a a product catalog with information about all the products being sold, including categories, prices, and stock levels.',
        'inputSchema': {
            'json': {
                'type': 'object',
                'properties': {
                    'category': {
                        'type': 'string',
                        'description': 'Retrieves product information data for all products in this category.'
                    }
                },
                'required': [
                    'category'
                ]
            }
        }
    }
})

Now let's define the user request.  

In our example we'll use: "Can you provide a sales summary for 29th September 2023, and also give me the details of all products in the 'Electronics' category that were sold that day, including their prices and stock levels?"

Only a langage model with Tool Use can answer this request: it requires looking up information in the right external tools (step 2), and then providing a final answer based on the tool results (step 4).

In [ ]:
# preamble containing instructions about the task and the desired style for the output.

additional_model_fields ={"preamble": """
## Task & Context
You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging.

## Style Guide
Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling.
"""
}
# user request
input_text = "Can you provide a sales summary for 29th September 2023, and also give me some details about the products in the 'Electronics' category, for example their prices and stock levels?"


### Step 4 – the model smartly decides which tool(s) to use and how
The model intelligently selects the right tool(s) to call -- and the right parameters for each tool call -- based on the content of the user message.

In [ ]:
#define the converse function that will call converse API from Bedrock
def converse(messages, tool_config,  model_id):
    response = bedrock_rt.converse(
        messages=messages,
        toolConfig=tool_config,
        additionalModelRequestFields=additional_model_fields,
        modelId = model_id
    )
    return response

messages =[]
messages.append({
        "role": "user",
        "content": [{"text": input_text}]
    }
)

response = converse(messages, tool_config, model_id)

print("The model recommends doing the following tool calls:")
for content in response['output']['message']['content']:
    if 'toolUse' in content:
        print("\n")
        print(content['toolUse'])

### Step 5 – the tool calls are executed

You can then execute the appropriate calls, using the tool calls and tool parameters generated by the model.  
These tool calls return tool results that will be fed to the model in Step 4.

In [ ]:
tool_results = []
# Iterate over the tool calls generated by the model
for content in response['output']['message']['content']:
    if 'toolUse' in content:
        tool_call = content['toolUse']
        # here is where you would call the tool recommended by the model, using the parameters recommended by the model
        # Get the function name from the tool_call dictionary
        print(f"= running tool {tool_call['name']}, with parameters: {tool_call['input']}")
    
        output = functions_map[tool_call['name']](*tool_call['input'])
        outputs = [output]
        print(outputs)
        print(f"== tool results: {outputs}")
        # store your tool results in this format
        tool_results.append({
            "role": "user",
            "content": [
                {
                'toolResult': {
                    'toolUseId':tool_call['toolUseId'],
                    'content': [
                        {
                            "text": output
                        }
                    ]
                }
                }
            ]
        
        })

print("Tool results that will be fed back to the model in step 4:")
print(json.dumps(tool_results, indent=4))

### Step 6 - the model generates a final answer based on the tool results
Finally, the developer calls the Cohere model, providing the tools results, in order to generate the model's final answer.  

Bonus: At Cohere, all Tool Use calls come with... **precise citations**! 🎉
The model cites which tool results were used to generate the final answer.  
These citations make it easy to check where the model’s generated response claims are coming from.  
They help users gain visibility into the model reasoning, and sanity check the final model generation.  
These citations are optional — you can decide to ignore them.



In [ ]:
def converse_with_tools(messages,  model_id):
    response = bedrock_rt.converse(
        messages=messages,
        additionalModelRequestFields=additional_model_fields,
        inferenceConfig = {
            "temperature": 0.3
        },
        modelId = model_id
    )
    return response


response = converse_with_tools(messages, model_id)

In [ ]:
print("Final answer:")
print(response['output']['message']['content'][0]['text'])

---
## Conclusion

Tool use opens up a wide range of new use cases. Here are a few examples:

- **Function calling**: It's now possible to ask the model to output a JSON object with specific function parameters.
For instance, this allows your chatbot to interact with your CRM to change the status of a deal, or to engage with a Python interpreter to conduct data science analyses.

- **Query transformation**: You can transform a user message into a search query for a vector database or any search engine.
For instance, this enables your work assistant to automatically retrieve the appropriate data from your company's documentation by creating the right query for your vector database.

- **Advanced searches**: You can transform a user message into one-or-many queries, to do multiple subtasks based on the content of the message.
For instance, this allows your chatbot to search across different databases and platforms to retrieve relevant information or to conduct comparative analysis.

